## **Text Classification**

In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Text Classification/train_data.csv', delimiter=' ')

In [5]:
df.shape

(25000, 3)

In [6]:
df.head()

,sentence,sentiment,polarity
0,bromwell high cartoon comedy ran time some pr...,9,1
1,homelessness or houselessness george carlin s...,8,1
2,brilliant over acting lesley ann warren best ...,10,1
3,easily underrated film inn brooks cannon sure...,7,1
4,is typical mel brooks film much less slapstic...,8,1


In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['sentence'])

thresh=35

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value

common = tot_cnt-cnt

tokenizer = Tokenizer(num_words = common)
tokenizer.fit_on_texts(df['sentence'])
word2idx = tokenizer.word_index

In [9]:
len_word2idx = common
len_word2idx

9253

In [10]:
sequences = tokenizer.texts_to_sequences(df['sentence'])

In [11]:
X_train = pad_sequences(sequences, padding='post', maxlen=200)
Y_train = []
for i in range(25000):
    if i < 12500:
        Y_train.append(1)
    else:
        Y_train.append(0)
Y_train = np.reshape(Y_train, (len(Y_train),)+(1,))

print(X_train.shape)
print(Y_train.shape)

(25000, 200)
(25000, 1)


In [21]:
test_df = pd.read_csv('/content/drive/MyDrive/Text Classification/test_data.csv', delimiter=' ')

test_df['sentence'].pop(11663)
tokenizer_test = Tokenizer(num_words = common)
tokenizer_test.fit_on_texts(test_df['sentence'])
word2idx_test = tokenizer_test.word_index
sequences_test = tokenizer_test.texts_to_sequences(test_df['sentence'])

X_test = pad_sequences(sequences_test, padding='post', maxlen=200)

Y_test = []
for i in range(25000):
    if i < 12500:
        Y_test.append(1)
    else:
        Y_test.append(0)
Y_test.pop(11663)       
Y_test = np.reshape(Y_test, (len(Y_test),)+(1,))


print(X_test.shape)
print(Y_test.shape)

(24999, 200)
(24999, 1)


In [22]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import random
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers

In [23]:
input_X = keras.Input(shape=(200,))

X = layers.Embedding(len_word2idx, 128)(input_X)
X = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(X)
X = layers.Bidirectional(layers.LSTM(64))(X)

out = layers.Dense(1, activation='sigmoid')(X)

model = keras.Model(input_X, out)

In [24]:
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 200, 128)          1184384   
_________________________________________________________________
bidirectional (Bidirectional (None, 200, 128)          98816     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,382,145
Trainable params: 1,382,145
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [26]:

model.fit(X_train, Y_train, epochs=5, batch_size = 64)

Epoch 1/5
391/391 [==============================] - 239s 596ms/step - loss: 0.4817 - accuracy: 0.7505
Epoch 2/5
391/391 [==============================] - 234s 598ms/step - loss: 0.2088 - accuracy: 0.9236
Epoch 3/5
391/391 [==============================] - 234s 599ms/step - loss: 0.1388 - accuracy: 0.9523
Epoch 4/5
391/391 [==============================] - 235s 601ms/step - loss: 0.0922 - accuracy: 0.9690
Epoch 5/5
391/391 [==============================] - 235s 601ms/step - loss: 0.0894 - accuracy: 0.9696


In [27]:
train_loss, train_acc = model.evaluate(X_train, Y_train)
print("train_loss : ", train_loss)
print("train_acc : ", train_acc)

782/782 [==============================] - 66s 83ms/step - loss: 0.0548 - accuracy: 0.9855
train_loss :  0.05484621599316597
train_acc :  0.985480010509491


In [28]:
pred_train = model.predict(X_train)

In [29]:
test_loss, test_acc = model.evaluate(X_test, Y_test)
print("test_loss : ", test_loss)
print("test_acc : ", test_acc)

782/782 [==============================] - 64s 82ms/step - loss: 1.7103 - accuracy: 0.5262
test_loss :  1.7102558612823486
test_acc :  0.5262210369110107


In [30]:
pred_test = model.predict(X_test)